In [1]:
import pandas as pd
from pathlib import Path
import os

In [3]:
DATA_DIR = Path(os.environ.get("DATA_DIR"))
LANDING_DIR = DATA_DIR / "landing"
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(exist_ok=True, parents=True)
xlsx_files = list(LANDING_DIR.glob("RO etoh.xlsx"))
landing_file = xlsx_files[0]

In [8]:
df_master = pd.read_excel(
    landing_file,
    sheet_name="Sheet1",
    header=1,
)
df_master = df_master.rename(columns={"ID#": "mouse_name"})
df_master.head()

,mouse_name,Video ID#,Week 1,Week 2,Week 3,Total,Drink,Eat,Groom,Hang,...,Groom.3,Hang.3,Sniff.3,Rear.3,Rest.3,Walk.3,Paw eat.3,Exp.3,Con.3,Low.3
0,E1,cam_17202346,4.495570,3.969168,6.162342,14.627080,0.003119,0.129735,0.094326,0.029656,...,0.091550,0.006149,0.434660,0.009885,0.246816,0.027027,0.082441,0.043061,0.275462,0.681477
1,E2,cam_17202341,1.146024,3.159573,4.223275,8.528872,0.001732,0.077949,0.139415,0.017598,...,0.093653,0.008457,0.330603,0.008864,0.443636,0.011104,0.049146,0.028425,0.197336,0.774239
2,E3,cam_17202345,3.575861,5.751440,6.471311,15.798613,0.003574,0.135989,0.120115,0.020008,...,0.115884,0.006336,0.396861,0.021844,0.264954,0.034265,0.053278,0.062445,0.275740,0.661815
3,E4,cam_17202339,1.701235,3.624732,4.582490,9.908457,0.008230,0.182438,0.229743,0.008081,...,0.098150,0.006394,0.250427,0.004297,0.497015,0.007269,0.063372,0.017960,0.234599,0.747441
4,E5,cam_17202342,0.007291,2.630249,4.027456,6.664996,0.001470,0.096349,0.153760,0.016484,...,0.080388,0.010740,0.299671,0.008078,0.480246,0.011729,0.039435,0.030547,0.189536,0.779917


In [10]:
mouse_cols = ["mouse_name", "Video ID#", "Video ID#"]

df_mice = df_master.loc[:, mouse_cols]
df_mice = df_mice.rename(
    columns={
        "Video ID#": "video_id",
    }
)
display(df_mice.head(3))

df_mice.to_csv(RAW_DIR / "etoh_mice.csv", index=False)

,mouse_name,video_id,video_id
0,E1,cam_17202346,cam_17202346
1,E2,cam_17202341,cam_17202341
2,E3,cam_17202345,cam_17202345


In [11]:
behavior_cols = ["mouse_name", "Week 1", "Week 2", "Week 3"]
df_behavior = df_master.loc[:, behavior_cols]
df_behavior = df_behavior.rename(
    columns={
        "Week 1": "week_1",
        "Week 2": "week_2",
        "Week 3": "week_3",
    }
)
display(df_behavior.head(3))

df_behavior.to_csv(RAW_DIR / "etoh_behavior.csv", index=False)

,mouse_name,week_1,week_2,week_3
0,E1,4.495570,3.969168,6.162342
1,E2,1.146024,3.159573,4.223275
2,E3,3.575861,5.751440,6.471311


In [12]:
MOUSE_NAME_DF = df_mice[["mouse_name"]]

display(MOUSE_NAME_DF.head(3))

,mouse_name
0,E1
1,E2
2,E3


In [14]:
AGG_MAPPER = dict(con="agg_cons", exp="agg_explore", low="agg_low")

In [17]:
df_base = pd.read_excel(landing_file, sheet_name="Sheet1", header=1, usecols="G:R")

df_base.columns = df_base.columns.str.lower().str.replace(" ", "_")
df_base = df_base.rename(columns=AGG_MAPPER)
df_base = pd.concat([MOUSE_NAME_DF, df_base], axis=1)

display(df_base.head(3))

df_base.to_csv(RAW_DIR / "etoh_hc_base.csv", index=False)

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,paw_eat,agg_explore,agg_cons,agg_low
0,E1,0.003119,0.129735,0.094326,0.029656,0.231052,0.021653,0.375814,0.047309,0.067335,0.098617,0.294516,0.606867
1,E2,0.001732,0.077949,0.139415,0.017598,0.194011,0.022527,0.477696,0.024578,0.044493,0.064703,0.263590,0.671707
2,E3,0.003574,0.135989,0.120115,0.020008,0.229322,0.032546,0.357208,0.052183,0.049056,0.104737,0.308733,0.586530


In [19]:
df_week1 = pd.read_excel(landing_file, sheet_name="Sheet1", header=1, usecols="S:AD")

df_week1.columns = df_week1.columns.str.lower().str.replace(" ", "_")

column_mapper = {col: f"{col.split('.')[0]}" for col in df_week1.columns if "." in col}

df_week1 = df_week1.rename(columns=column_mapper)
df_week1 = df_week1.rename(columns=AGG_MAPPER)
df_week1 = pd.concat([MOUSE_NAME_DF, df_week1], axis=1)

display(df_week1.head(3))

df_week1.to_csv(RAW_DIR / "etoh_hc_week1.csv", index=False)

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,paw_eat,agg_explore,agg_cons,agg_low
0,E1,0.003008,0.128474,0.087622,0.013271,0.337673,0.012953,0.325045,0.032277,0.059675,0.058502,0.278780,0.662718
1,E2,0.001486,0.063569,0.107053,0.009114,0.247254,0.012959,0.484136,0.014637,0.059792,0.036710,0.231900,0.731390
2,E3,0.002052,0.140107,0.103560,0.009736,0.305794,0.024435,0.312634,0.035023,0.066658,0.069195,0.312377,0.618428


In [20]:
df_week2 = pd.read_excel(landing_file, sheet_name="Sheet1", header=1, usecols="AE:AP")

df_week2.columns = df_week2.columns.str.lower().str.replace(" ", "_")

column_mapper = {col: f"{col.split('.')[0]}" for col in df_week2.columns if "." in col}

df_week2 = df_week2.rename(columns=column_mapper)
df_week2 = df_week2.rename(columns=AGG_MAPPER)
df_week2 = pd.concat([MOUSE_NAME_DF, df_week2], axis=1)

display(df_week2.head(3))

df_week2.to_csv(RAW_DIR / "etoh_hc_week2.csv", index=False)

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,paw_eat,agg_explore,agg_cons,agg_low
0,E1,0.002235,0.105357,0.098703,0.007094,0.379578,0.010221,0.309040,0.021783,0.065988,0.039098,0.272283,0.688618
1,E2,0.000964,0.056864,0.107676,0.004836,0.262697,0.008521,0.488497,0.012016,0.057928,0.025374,0.223431,0.751194
2,E3,0.001817,0.139061,0.111180,0.007840,0.359681,0.025803,0.255253,0.037509,0.061856,0.071152,0.313913,0.614934


In [21]:
df_week3 = pd.read_excel(landing_file, sheet_name="Sheet1", header=1, usecols="AQ:BB")

df_week3.columns = df_week3.columns.str.lower().str.replace(" ", "_")

column_mapper = {col: f"{col.split('.')[0]}" for col in df_week3.columns if "." in col}

df_week3 = df_week3.rename(columns=column_mapper)
df_week3 = df_week3.rename(columns=AGG_MAPPER)
df_week3 = pd.concat([MOUSE_NAME_DF, df_week3], axis=1)

display(df_week3.head(3))

df_week3.to_csv(RAW_DIR / "etoh_hc_week3.csv", index=False)

,mouse_name,drink,eat,groom,hang,sniff,rear,rest,walk,paw_eat,agg_explore,agg_cons,agg_low
0,E1,0.001815,0.099656,0.091550,0.006149,0.434660,0.009885,0.246816,0.027027,0.082441,0.043061,0.275462,0.681477
1,E2,0.001130,0.053408,0.093653,0.008457,0.330603,0.008864,0.443636,0.011104,0.049146,0.028425,0.197336,0.774239
2,E3,0.003336,0.103242,0.115884,0.006336,0.396861,0.021844,0.264954,0.034265,0.053278,0.062445,0.275740,0.661815
